# 顔文字生成器（サンプル）
単純ニューラルネットワークだけのシンプル構成

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

## 前処理

### データ読み込み

In [2]:
KAOMOJI_MAX = 20    # 顔文字最大長

kaomoji_list = []   # 顔文字リスト
char_list = []      # 顔文字に使用されている文字のリスト

pad = '<pad>'       # 文字列の長さを一定にするためのダミー文字
char_list.append(pad)

file_name = 'kaomoji_MAX=' + str(KAOMOJI_MAX) + '.txt'

with open(file_name, mode='r') as file:
  for line in file:
    temp = list(line.replace('\n', ''))
    temp += [pad for _ in range(KAOMOJI_MAX - len(temp))]
    kaomoji_list.append(temp)
    char_list += temp

# 重複を消す
char_list = sorted(set(char_list), key=char_list.index)

In [3]:
char_list.index(pad)

0

In [4]:
print('Number of kaomoji  :', len(kaomoji_list))
print('Number of character:', len(char_list))

Number of kaomoji  : 11587
Number of character: 1103


### 添字検索
顔文字に使われる文字が文字リストの何番目にあるか調べる

In [5]:
kaomoji_index = []    # 添字リスト

for data in kaomoji_list:
  temp = [char_list.index(c) for c in data]
  kaomoji_index.append(temp)

In [6]:
print(kaomoji_index[0])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 0, 0, 0, 0, 0, 0, 0]


### One-hotベクトル化

In [7]:
kaomoji_num = len(kaomoji_index)
kaomoji_size = len(kaomoji_index[0])
char_num = len(char_list)

# One-hotベクトルリスト
one_hot_data = np.zeros((kaomoji_num, kaomoji_size, char_num))

for i, index in enumerate(kaomoji_index):
  mask = range(char_num) == np.array(index).reshape((kaomoji_size, 1))
  one_hot_data[i][mask] = 1

In [8]:
one_hot_data.shape

(11587, 20, 1103)

### 訓練・検証・テスト用に分ける

In [9]:
train_num = 10000
valid_num = 1000

np.random.shuffle(one_hot_data)
dataset = torch.tensor(one_hot_data.astype('float32'))
dataset_train = dataset[:train_num]
dataset_valid = dataset[train_num:train_num+valid_num]
dataset_test  = dataset[train_num+valid_num:]

In [10]:
dataset.shape

torch.Size([11587, 20, 1103])

## 学習

### ネットワークの定義

In [11]:
class SimpleNet(nn.Module):
  def __init__(self, in_dim, hid_dim):
    super().__init__()
    self.encoder = nn.Linear(in_dim, hid_dim)
    self.decoder = nn.Linear(hid_dim, in_dim)

  def forward(self, x):
    x = self.encoder(x)
    return self.decoder(x)

### 学習

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
n_epochs = 5
hid_dim = 30

kaomoji_num, kaomoji_size, char_num = one_hot_data.shape
net = SimpleNet(char_num, hid_dim)

optimizer = optim.Adam(net.parameters())    # 最適化
criterion = nn.CrossEntropyLoss()           # 損失関数

In [16]:
for epoch in range(n_epochs):
  losses_train = []
  losses_valid = []

  # 訓練
  net.train()
  for x in dataset_train:
    net.zero_grad()  # 勾配の初期化

    #x.to(device)
    #t.to(device)

    y = net(x)

    loss = criterion(y, x)
    loss.backward()  # 誤差の逆伝播
    losses_train.append(loss.tolist())

    optimizer.step()  # パラメータの更新

  # 検証
  net.eval()
  for x in dataset_valid:
    y = net(x)

    loss = criterion(y, x)
    losses_valid.append(loss.tolist())

  if (epoch+1) % 1 == 0:
    print('EPOCH: {:>2}, Train Loss: {:>4.5f}, Valid Loss: {:>4.5f}'.format(
        epoch,
        np.mean(losses_train),
        np.mean(losses_valid),
    ))

EPOCH:  0, Train Loss: 0.60343, Valid Loss: 0.14477
EPOCH:  1, Train Loss: 0.11416, Valid Loss: 0.07889
EPOCH:  2, Train Loss: 0.06524, Valid Loss: 0.06071
EPOCH:  3, Train Loss: 0.04514, Valid Loss: 0.05214
EPOCH:  4, Train Loss: 0.03416, Valid Loss: 0.04702


## 評価

In [17]:
def convert_str(c_list):
  c_list = np.array(char_list)[c_list.argmax(dim=1)]
  c_list = [c for c in c_list if c != '<pad>']

  return ''.join(c_list)

def generate(base, net, rate=3.0):
  eps = 2*rate * np.random.rand(kaomoji_size, hid_dim).astype('float32') - rate

  z = net.encoder(base) + torch.tensor(eps)
  y = net.decoder(z)

  gen = convert_str(y)

  return ''.join(gen)

In [21]:
for i in np.random.randint(0, len(dataset_test), size=10):
  test = dataset_test[i]
  print('base     :', convert_str(test))
  print('generate :', generate(test, net))

base     : ~(^◇^)/
generate : ~(^◇▿)/ຶ⊖
base     : (*^-^)
generate : (*^-^)
base     : (@。@)/!?
generate : (@。ペ)/!?☀
base     : ((_-　)(　-。))、
generate : ((_-　)(　-。))、
base     : (●　́ノω`)♪
generate : (●　́ノॢ`)♪
base     : ━━━━((゚Д゚)━━━━
generate : ━━━━((゚Д゚)バ━━━
base     : (　*^3^)ノ、・・
generate : (　*^3^)ノ、・・
base     : ꒰✩’ω`ૢ✩꒱
generate : よж’ω`жш꒱░
base     : Ψ(　́д`)Ψ・・・
generate : y(　́у`)◇・・・
base     : (　̄ー　̄(。-_-。*)ゝ
generate : (　̄ー╰̄(。-͟-。*)ゝ


In [25]:
def similar(c, net, num=5):
  index = char_list.index(c)
  weight = net.encoder(torch.tensor(np.eye(char_num, dtype='float32')))
  vector = weight[index]
  diff = weight - vector
  norm = torch.norm(diff, dim=1).detach().numpy()
  for _ in range(num):
    min_index = norm.argmin()
    print('{} : {}'.format(char_list[min_index], norm[min_index]))
    norm[min_index] = 100

In [26]:
similar('(', net)

( : 0.0
X : 13.01224422454834
̅ : 13.625532150268555
¬ : 13.712963104248047
∫ : 13.740323066711426


## 感想

このネットワークは，  
Encoder：1層のニューラルネットワーク（$ Wx+b $ のやつ）  
Decoder：1層のニューラルネットワーク（$ Wx+b $ のやつ）  
のシンプルな構成

入力：(＾ー＾) としたらとき  
出力：(＾ー＾) となるからできているように見えるが，  
実体は，1文字に対して1文字が出ているだけ．  
要は，顔文字を分解した1文字ずつが，  
入力："（"　　------->　　出力："（"  
入力："＾"　　------->　　出力："＾"  
入力："ー"　　------->　　出力："ー"  
入力："＾"　　------->　　出力："＾"  
入力："）"　　------->　　出力："）"  
となっているだけで，顔文字でなくとも再現可能である．

1文字に対して固有の潜在ベクトルがある．
文字をOne-hotベクトルに変換してから入力しているため，入力の次元は大きくなるが，オートエンコーダを通して次元圧縮はできている．

また，1文字の潜在ベクトル同士の距離を計算すると，例えば「(」であれば一番近いものは「)」となり，なんとなくできている感じもある．

